## Align reads to genome using `HISAT2`

In [ ]:
REF_DIR="/home/ilya/data1/ref/Caenorhabditis_elegans"
BASE_DIR="../data"
INDEX="$REF_DIR/wbcel235/genome"

hisat_exe="$HOME/bin/hisat2-2.1.0/hisat2"

for sample in $(ls $BASE_DIR | grep ^ig)
do
    sample_dir="$BASE_DIR/$sample/trim"
    result_dir="../results/$sample"
    
    if [ ! -d "$result_dir" ]; then
        echo "Creating $result_dir ..."
        mkdir -p $result_dir
    fi
    
    echo "Unzipping reads for $sample ..."
    read1=$sample_dir/read1.fifo
    read2=$sample_dir/read2.fifo
    se1=$sample_dir/se1.fifo
    se2=$sample_dir/se2.fifo
    mkfifo $read1
    mkfifo $read2
    mkfifo $se1
    mkfifo $se2
    
    zcat $sample_dir/R1_pe.fastq.gz > $read1 &
    zcat $sample_dir/R2_pe.fastq.gz > $read2 &
    zcat $sample_dir/R1_se.fastq.gz > $se1 &
    zcat $sample_dir/R2_se.fastq.gz > $se2 &
    
    $hisat_exe -p 30 --dta \
        -x $INDEX \
        -1 $read1 -2 $read2 -U $se1,$se2 \
    | samtools view -bhS - > "$result_dir/$sample.bam"
    
    rm $sample_dir/*.fifo 
    
done

In [ ]:
rm ../data/ig01/trim/*.fifo
ls -lah ../data/ig01/trim

## Untrimmed reads

In [ ]:
REF_DIR="/home/ilya/data1/ref/Caenorhabditis_elegans"
BASE_DIR="../data"
INDEX="$REF_DIR/wbcel235/genome"

hisat_exe="$HOME/bin/hisat2-2.1.0/hisat2"

for sample in $(ls $BASE_DIR | grep ^ig)
do
    sample_dir=$BASE_DIR/$sample
    result_dir="../results/$sample"
    
    if [ ! -d "$result_dir" ]; then
        echo "Creating $result_dir ..."
        mkdir -p $result_dir
    fi
    
    echo "Unzipping reads for $sample ..."
    read1=$sample_dir/read1.fifo
    read2=$sample_dir/read2.fifo
    mkfifo $read1
    mkfifo $read2
    
    zcat $sample_dir/R1.fastq.gz > $read1 &
    zcat $sample_dir/R2.fastq.gz > $read2 &
    
    $hisat_exe -p 30 --dta --quiet \
        -x $INDEX \
        -1 $read1 -2 $read2 \
    | samtools view -bhS - > "$result_dir/$sample.bam"
    
    rm $sample_dir/read*.fifo 
    
done

In [ ]:
for sample in $(ls $BASE_DIR | grep ^ig)
do
    result_dir="../results/$sample"
    samtools sort -@ 30 -o "$result_dir/$sample.sorted.bam" "$result_dir/$sample.bam"
    samtools index -@ 30 "$result_dir/$sample.sorted.bam"
done

## Assemble transcripts and estimate abundances

In [ ]:
GENES="$REF_DIR/Ensembl/WBcel235/Annotation/Genes/genes.gtf"
stringtie_exe="$HOME/bin/stringtie-1.3.3b.Linux_x86_64/stringtie"

for sample in $(ls $BASE_DIR | grep ^ig)
do
    result_dir="../results/$sample"
    $stringtie_exe -p 30 -G $GENES \
        -o "$result_dir/$sample.gtf" \
        -l $sample "$result_dir/$sample.sorted.bam"
done

In [ ]:
$stringtie_exe --merge -p 30 -G $GENES -o "../ref/merged_KNAC.gtf" "../ref/mergelist_KNAC.txt"

In [ ]:
$stringtie_exe --merge -p 30 -G $GENES -o "../ref/merged_CAci.gtf" "../ref/mergelist_CAci.txt"

In [ ]:
for i in $(seq -f "%02g" 1 6)
do
    sample="ig$i"
    result_dir="../results/$sample"
    $stringtie_exe -e -B -p 30 -G "../ref/merged_KNAC.gtf" \
        -o "$result_dir/$sample.gtf" \
        "$result_dir/$sample.sorted.bam"
done

In [ ]:
for i in $(seq -f "%02g" 7 12)
do
    sample="ig$i"
    result_dir="../results/$sample"
    $stringtie_exe -e -B -p 30 -G "../ref/merged_CAci.gtf" \
        -o "$result_dir/$sample.gtf" \
        "$result_dir/$sample.sorted.bam"
done